# Final project - script

W ramach zadania zaliczeniowego przygotowałem bibliotekę do przetwarzania danych o dochodach PIT w jednostkach terytorialnych Polski.

In [ ]:
%pip install -e calculate_income --quiet

In [ ]:
import configparser
import os
import pandas as pd
import numpy as np
from IPython.display import display
import calculate_income as cal_income
import warnings
warnings.filterwarnings('ignore')
%load_ext line_profiler

In [ ]:
dir_name = os.path.dirname("script.ipynb")
config = configparser.ConfigParser()
config.read(os.path.join(dir_name, "config.ini"))

W celu określenia rozdziału klasyfikacji finansowej odpowiedzialnego za PIT użyłem przepisów na rok 2021 pokrywające się w tym zakresie z poprzednimi klasyfikacjami.

Paragraf i rozdział określają rodzaj dochodu ze względu na źródło pochodzenia i rodzaj dochodu:

0010 Wpływy z podatku dochodowego od osób fizycznych\
75621 Udziały gmin w podatkach stanowiących dochód budżetu państw\
75622 Udziały powiatów w podatkach stanowiących dochód budżetu państwa\
75623 Udziały województw w podatkach stanowiących dochód budżetu państwa\
75634 Udziały związków metropolitalnych w podatku dochodowym od osób fizycznych\

WK kod województwa\
PK kod powiatu\
GK kod gminy\
GT kod typu gminy (1 miejska, 2 wiejska, 3 miejsko-wiejska)

### Opodatkowany dochód dla jednostek samorządów terytorialnych z roku 2020 oraz 2019.

In [ ]:
df_gm = cal_income.income_comparison(config["data"]["y1_f1"], config["data"]["y2_f1"])
df_pow = cal_income.income_comparison(config["data"]["y1_f2"], config["data"]["y2_f2"])
df_mnpp = cal_income.income_comparison(config["data"]["y1_f3"], config["data"]["y2_f3"])
df_pow_mnpp = pd.concat([df_pow, df_mnpp])
df_metrop = cal_income.income_comparison(config["data"]["y1_f4"], config["data"]["y2_f4"])
df_woj = cal_income.income_comparison(config["data"]["y1_f5"], config["data"]["y2_f5"])

In [ ]:
%%time
display(df_gm.head())
display(df_pow_mnpp.head())
display(df_metrop)
display(df_woj.head())

Zgodnie z wynikami dochody w jednostkach terytorialnych nieznacznie obniżyły się względem poprzedniego roku.

### Średni dochód opodatkowany dla poszczególnych jednostek samorządu terytorialnego.

Następnie połączyłem te dane z danymi o ludności w poszczególnych Jednostkach Samorządu Terytorialnego i policzyłem średni dochód opodatkowany dla tych jednostek. Wykonanie tych poleceń zajmuje 254 ms, 48.4 ms, 36.8 ms. Różnica wyniku dla gmin związana jest z większą obróbką danych.

In [ ]:
%%time
df_gm_mean = cal_income.mean_inc_gm(df_gm, config["data"]["gm_lud"])
display(df_gm_mean.head())

In [ ]:
%%time
df_pow_mnpp_mean = cal_income.mean_inc_pow(df_pow, config["data"]["pow_lud"])
display(df_pow_mnpp_mean.head())

In [ ]:
%%time
df_woj_mean = cal_income.mean_inc_woj(df_woj, config["data"]["woj_lud"])
display(df_woj_mean.head())

Dla Powiatów i Województw policzyłem wariancję dochodu w podległych jednostkach samorządu terytorialnego i porównałem przewidziany dochód dla tej jednostki ze średnią ważoną jednostek podległych.

In [ ]:
df_gm_var = cal_income.income_variance_gm(df_gm_mean, df_pow_mnpp_mean)
display(df_gm_var.head(15))

W powiatach, w których jest małe zróżnicowanie dochodu pomiędzy gminami średnia dochodu w gminach jest bliska dochodowi w powiecie. W powiatach, w których jest duże zróżnicowanie dochodu pomiędzy gminami obserwuję, że średni dochód w powiecie różni się od średniej ze średnich dochodów w gminach.

In [ ]:
%%time
df_gm_pow_mnpp = pd.concat([df_pow_mnpp_mean, df_gm_mean])
df_pow_var = cal_income.income_variance_pow(df_gm_pow_mnpp, df_woj_mean)
display(df_pow_var)

W przypadku województwa te różnice wyrównują się. Mimo wysokich zróżnicowań pomiędzy dochodami podległych jednostek, średnia z tych dochodów w większości przypadków jest zbliżona do dochodu w danym województwie.

In [ ]:
%%time
df = [df_gm, df_pow, df_mnpp, df_metrop, df_woj]
df_all_income = pd.concat(df)

In [ ]:
%%time
with pd.ExcelWriter(config["data"]["results"]) as writer:
    df_all_income.to_excel(writer, sheet_name='doch-jst')
    df_gm_mean.to_excel(writer, sheet_name='gm-srednie-doch')
    df_pow_mnpp_mean.to_excel(writer, sheet_name='pow-mnpp-srednie-doch')
    df_woj_mean.to_excel(writer, sheet_name='woj-srednie-doch')
    df_gm_var.to_excel(writer, sheet_name='pow-stat')
    df_pow_var.to_excel(writer, sheet_name='woj-stat')

Wyniki zapisuję w arkuszach Excel w folderze “results”. Ta operacja zajmuje 2.06 s.

## Profilowanie

Badam funkcję w skrypcie, której wykonanie zajmuje najwięcej czasu.

In [ ]:
%lprun -f cal_income.income_comparison cal_income.income_comparison(config["data"]["y1_f1"], config["data"]["y2_f1"])

In [ ]:
%lprun -f cal_income.income_jst cal_income.income_jst(config["data"]["y1_f1"])

In [ ]:
%lprun -f cal_income.load cal_income.load(config["data"]["y1_f1"])

Wynik profilowania wykazuje, iż najwięcej czasu funkcja poświęca pobraniu danych z pliku.